# Workflow Examples: Four Patterns

This chapter demonstrates four common workflow patterns using the nodes from the previous chapter. Import the pre-built workflows, run them, and observe how the nodes work together.

**Note on credentials:** These workflows require API credentials for an AI provider (OpenRouter, OpenAI, or Google). If you do not have credentials set up, you can still explore the workflow structure and understand what each node would do.

---

## How to Import and Run a Workflow

Each workflow in this chapter provides two ways to import it:

| Option | When to use |
|--------|-------------|
| **Import via URL** | Fastest method — just copy the URL and paste it in n8n |
| **Download** | If you prefer to save the file locally first |

**Import from URL (recommended):**
1. Copy the URL from the workflow section (starts with `https://raw.githubusercontent.com/...`)
2. In n8n, click **Workflows** → **Add Workflow**
3. Click the **three-dot menu (⋮)** in the top-right
4. Select **Import from URL**
5. Paste the URL and click **Import**
6. Click **Save**

**Import from File:**
1. Click the **Download** link to save the `.json` file
2. In n8n, click **Workflows** → **Add Workflow**
3. Click the **three-dot menu (⋮)** in the top-right
4. Select **Import from File**
5. Choose the downloaded `.json` file
6. Click **Save**

**Run:**
1. Open the workflow in the editor
2. Make sure credentials are set up (Settings → Credentials)
3. Click **Execute Workflow** in the top toolbar
4. Click any node to see its output in the right panel

**Cost safety:** Keep workflows **Inactive** (toggle OFF) while learning. After the first successful AI call, **Pin data** (📌) on that node before editing downstream nodes — this prevents repeated API charges.

---

## Pattern 1: Prompt Chaining

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Manual Trigger │────▶│   Edit Fields   │────▶│  Basic LLM #1   │────▶│  Basic LLM #2   │──...
└─────────────────┘     └─────────────────┘     └─────────────────┘     └─────────────────┘
```

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/01_prompt_chaining.json
> ```
>
> **Download:** {download}`01_prompt_chaining.json <_static/workflows/01_prompt_chaining.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Prompt Chaining Practice"

### Step 2: Add the trigger

1. Click the **+** button on the canvas
2. Search for **Manual Trigger** → click to add
3. This starts your workflow when you click "Execute Workflow"

### Step 3: Add input data (Edit Fields)

1. Click **+** on the right side of Manual Trigger
2. Search for **Edit Fields** (or "Set") → add it
3. Rename the node to `Input — Writing Brief`
4. Add three String fields:

| Name | Value |
|------|-------|
| `topic` | `Create a short internal memo: Why we should pilot an AI agent for customer support.` |
| `audience` | `Operations and Customer Support leadership` |
| `constraints` | `Keep it under 400 words. Use clear, non-technical language. Include 3 bullet points with benefits and 1 with risks.` |

### Step 4: Add Step 1 — Create Outline (Basic LLM Chain)

1. Click **+** on the right side of Edit Fields
2. Search for **Basic LLM Chain** → add it
3. Rename to `Step 1 — Create Outline`
4. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt (User Message)** — click the field, switch to Expression mode with `{{ }}`:
     ```
     Topic: {{ $json.topic }}
     Audience: {{ $json.audience }}
     Constraints: {{ $json.constraints }}

     Return only the outline.
     ```
5. Scroll down to **Chat Messages** → click **Add Message**:
   - Type: `System`
   - Message:
     ```
     You are a writing assistant.
     Create a clear outline for the memo.

     Rules:
     - 5–7 sections
     - Include where the benefits bullets and risks bullet will go
     - Output ONLY the outline
     ```
6. Add the Chat Model:
   - Click **+ Chat Model** at the bottom of the node
   - Select **OpenRouter Chat Model** (or OpenAI, Google, etc.)
   - Choose your credential
   - Model: `openai/gpt-4o-mini` (or `deepseek/deepseek-chat-v3-0324:free`)

### Step 5: Add Store Outline (Edit Fields)

1. Click **+** on the right side of Step 1
2. Add **Edit Fields** → rename to `Store Outline`
3. Add one String field:
   - Name: `outline`
   - Value (Expression mode): `{{ $json.text }}`

### Step 6: Add Step 2 — Improve Outline (Basic LLM Chain)

1. Click **+** on the right side of Store Outline
2. Add **Basic LLM Chain** → rename to `Step 2 — Improve Outline`
3. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression mode):
     ```
     Original outline:
     {{ $json.outline }}

     Return only the improved outline.
     ```
4. Add System Message:
   ```
   You are a strict editor.
   Improve the outline using this checklist:
   - Logical flow
   - Clear headings
   - Covers constraints (benefits + risks)
   - Non-technical wording

   Output ONLY the improved outline (no commentary).
   ```
5. Connect the same Chat Model (it should auto-connect, or drag from the existing one)

### Step 7: Add Store Improved Outline (Edit Fields)

1. Click **+** → Add **Edit Fields** → rename to `Store Improved Outline`
2. Add field:
   - Name: `improved_outline`
   - Value (Expression): `{{ $json.text }}`

### Step 8: Add Step 3 — Draft Memo (Basic LLM Chain)

1. Click **+** → Add **Basic LLM Chain** → rename to `Step 3 — Draft Memo`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression):
     ```
     Outline:
     {{ $json.improved_outline }}

     Write the memo now.
     ```
3. Add System Message:
   ```
   You are a business writer.
   Write the memo based on the outline.

   Rules:
   - Follow the outline order
   - Keep under 400 words
   - Use clear, non-technical language
   - Include exactly: 3 benefit bullets + 1 risk bullet

   Output ONLY the memo text.
   ```

### Step 9: Add Output — Final Memo (Edit Fields)

1. Click **+** → Add **Edit Fields** → rename to `Output — Final Memo`
2. Add field:
   - Name: `final_memo`
   - Value (Expression): `{{ $json.text }}`
3. Enable **Keep Only Set** to clean up the output

### Step 10: Test your workflow

1. Click **Execute Workflow**
2. Click each node to see its output
3. Check the final memo in the Output node

**Tip:** After the first successful run, **Pin** the output of Step 1 to avoid API calls while you work on later steps.

::::

### Nodes in This Pattern

This pattern uses nodes you already know from Quick Start:

| Node | What it does |
|------|--------------|
| **Manual Trigger** | Starts the workflow when you click "Execute Workflow" |
| **Edit Fields (Set)** | Creates or transforms data fields |
| **Basic LLM Chain** | Sends a prompt to an AI model and returns the response |

The difference here: we chain **multiple** Basic LLM Chains in sequence, where each uses the previous one's output.

### What Problem This Solves

Breaking a complex writing task into smaller steps improves quality. Instead of asking an LLM to "write a memo" in one shot, you ask it to: (1) create an outline, (2) improve the outline, (3) write the final draft. Each step builds on the previous result.

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 250px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐     ┌──────────────────┐
│   Manual Trigger │────▶│ Input — Writing Brief  │────▶│  Step 1 — Outline      │────▶│   Store Outline  │
└──────────────────┘     └────────────────────────┘     └────────────────────────┘     └──────────────────┘
                                                                                                │
        ┌───────────────────────────────────────────────────────────────────────────────────────┘
        ▼
┌────────────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐
│  Step 2 — Improve      │────▶│   Store Improved       │────▶│  Step 3 — Draft Memo   │────▶│  Output — Final Memo   │
└────────────────────────┘     └────────────────────────┘     └────────────────────────┘     └────────────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Run: Prompt Chaining** | Manual Trigger | Starts the workflow |
| **Input — Writing Brief** | Set | Creates fields: `topic`, `audience`, `constraints` |
| **Step 1 — Create Outline** | Basic LLM Chain | Generates an outline → outputs `text` |
| **Store Outline** | Set | Saves `{{ $json.text }}` as `outline` |
| **Step 2 — Improve Outline** | Basic LLM Chain | Refines the outline using `{{ $json.outline }}` → outputs `text` |
| **Store Improved Outline** | Set | Saves `{{ $json.text }}` as `improved_outline` |
| **Step 3 — Draft Memo** | Basic LLM Chain | Writes final memo using `{{ $json.improved_outline }}` → outputs `text` |
| **Output — Final Memo** | Set | Saves `{{ $json.text }}` as `final_memo` (uses `keepOnlySet` to remove other fields) |

**Sub-node:** One `OpenRouter Chat Model` is shared by all three LLM Chain nodes (connected via dotted lines).

### Prompts Used

Each step uses a different **role** and **rules**. This is why chaining works better than one big prompt.

**Step 1 — Create Outline:**
```
System: You are a writing assistant.
Create a clear outline for the memo.

Rules:
- 5–7 sections
- Include where the benefits bullets and risks bullet will go
- Output ONLY the outline
```

**Step 2 — Improve Outline:**
```
System: You are a strict editor.
Improve the outline using this checklist:
- Logical flow
- Clear headings
- Covers constraints (benefits + risks)
- Non-technical wording

Output ONLY the improved outline (no commentary).
```

**Step 3 — Draft Memo:**
```
System: You are a business writer.
Write the memo based on the outline.

Rules:
- Follow the outline order
- Keep under 400 words
- Use clear, non-technical language
- Include exactly: 3 benefit bullets + 1 risk bullet

Output ONLY the memo text.
```

**Why this works:** Each step has a focused role (assistant → editor → writer), specific constraints, and ends with "Output ONLY..." to prevent extra commentary.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Writing Brief: { topic, audience, constraints }
    ↓
Step 1 LLM: { text: "1. Introduction..." }
    ↓
Store Outline: { outline: "1. Introduction..." }
    ↓
Step 2 LLM: { text: "1. Executive Summary..." }
    ↓
Store Improved: { improved_outline: "1. Executive Summary..." }
    ↓
Step 3 LLM: { text: "MEMO: Why We Should..." }
    ↓
Final Output: { final_memo: "MEMO: Why We Should..." }
```

### What to Observe

1. Click **Input — Writing Brief** → see the starting data
2. Click **Step 1 — Create Outline** → see the LLM's outline in `text`
3. Click **Store Outline** → see the same content now saved as `outline`
4. Follow this pattern through each step to see how data transforms

::::{dropdown} 🔍 Why do we need "Store" nodes? See the data transformation
:color: info

When a Basic LLM Chain runs, it **replaces** the previous data with only its output. This diagram shows what happens:

```
 STEP 1: Input                    STEP 2: LLM generates outline       STEP 3: Store saves it
┌─────────────────────┐          ┌─────────────────────┐             ┌─────────────────────┐
│ Input — Writing     │─────────▶│ Step 1 — Outline    │────────────▶│ Store Outline       │
│ Brief               │          │ (Basic LLM Chain)   │             │ (Edit Fields)       │
└─────────────────────┘          └─────────────────────┘             └─────────────────────┘
         │                                │                                   │
         ▼                                ▼                                   ▼
┌─────────────────────┐          ┌─────────────────────┐             ┌─────────────────────┐
│ {                   │          │ {                   │             │ {                   │
│   "topic": "AI",    │          │   "text":           │             │   "text":           │
│   "audience":       │          │     "1. Intro       │             │     "1. Intro...",  │
│     "doctors",      │          │      2. Benefits    │             │   "outline":        │
│   "constraints":    │          │      3. Risks..."   │             │     "1. Intro       │
│     "benefits,      │          │ }                   │             │      2. Benefits    │
│      risks"         │          │                     │             │      3. Risks..."   │
│ }                   │          │ ⚠️ topic, audience, │             │ }                   │
│                     │          │ constraints are     │             │                     │
│                     │          │ GONE!               │             │ 💡 Store adds       │
│                     │          │                     │             │ "outline" field     │
└─────────────────────┘          └─────────────────────┘             └─────────────────────┘
```

**Key insight:** 
- The LLM outputs only `{ "text": "..." }` — original fields are lost
- The Store node adds `outline` to save the result with a meaningful name
- Step 2 only needs `{{ $json.outline }}` so it works without the original fields
- This workflow is designed so each step only needs the previous step's output

**Why this pattern?** Each LLM step has a focused task. Step 2 doesn't need `topic` or `audience` — it only needs the outline to improve it.

::::

::::{dropdown} 🔍 How does the Switch node route data? See the branching
:color: info

The Switch node checks a field value and sends data to **only one** matching branch:

```
                         ┌─────────────────────────────────────────────┐
                         │ DATA BEFORE SWITCH                          │
                         │ {                                           │
                         │   "text": "refund",                         │
                         │   "route": "refund"                         │
                         │ }                                           │
                         │                                             │
                         │ ⚠️ Note: ticket_subject and ticket_body     │
                         │ are NOT here! They're accessed via          │
                         │ $node['Input — Support Ticket'].json        │
                         └─────────────────────────────────────────────┘
                                              │
                                              ▼
                                    ┌───────────────────┐
                                    │      SWITCH       │
                                    │                   │
                                    │  Checks: route    │
                                    │  Value: "refund"  │
                                    └───────────────────┘
                                     /        |        \
                                    /         |         \
                         "refund"  /    "order_status"   \  "support"
                                  /           |           \
                                 ▼            ▼            ▼
                    ┌──────────────────┐ ┌──────────────┐ ┌──────────────┐
                    │ ✅ HAS DATA      │ │ ❌ EMPTY     │ │ ❌ EMPTY     │
                    │                  │ │              │ │              │
                    │ {                │ │    { }       │ │    { }       │
                    │  "text":         │ │              │ │              │
                    │    "refund",     │ │  No match,   │ │  No match,   │
                    │  "route":        │ │  no data     │ │  no data     │
                    │    "refund"      │ │  flows here  │ │  flows here  │
                    │ }                │ │              │ │              │
                    └──────────────────┘ └──────────────┘ └──────────────┘
                            │
                            ▼
                    ┌──────────────────┐
                    │ Refund           │
                    │ Specialist LLM   │
                    │                  │
                    │ Uses expression: │
                    │ $node['Input —   │
                    │ Support Ticket'] │
                    │ .json.ticket_body│
                    └──────────────────┘
```

**Key insights:** 
- Only ONE branch receives data — the others are empty
- The original `ticket_subject` and `ticket_body` are NOT in the Switch data
- Specialist nodes "reach back" to get original data using `$node['Input — Support Ticket'].json`

::::

---

## Pattern 2: Routing

```
                                                 ┌─────────────────┐
                                            ┌───▶│ LLM: Refund     │
┌─────────────────┐     ┌─────────────────┐ │    └─────────────────┘
│  Manual Trigger │────▶│  LLM: Classify  │─┼───▶│ LLM: Order      │
└─────────────────┘     └─────────────────┘ │    └─────────────────┘
                                            └───▶│ LLM: Support    │
                                                 └─────────────────┘
```

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/02_routing.json
> ```
> 
> **Download:** {download}`02_routing.json <_static/workflows/02_routing.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Routing Practice"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Support Ticket`
3. Add two String fields:

| Name | Value |
|------|-------|
| `ticket_subject` | `Refund request — charged twice` |
| `ticket_body` | `Hi team,\nI was charged twice for order #A-10492.\nPlease refund the extra charge. I can provide a screenshot if needed.\nThanks,\nJamie` |

### Step 3: Add the Router (Basic LLM Chain)

1. Add **Basic LLM Chain** → rename to `Router — Choose Route`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression):
     ```
     Subject: {{ $json.ticket_subject }}
     Body:
     {{ $json.ticket_body }}

     Route label:
     ```
3. Add System Message:
   ```
   You route customer support tickets.

   Choose exactly ONE route label, lowercase, no punctuation:
   - refund
   - order_status
   - support

   Return ONLY the label.
   ```
4. Add your Chat Model

### Step 4: Add Store Route (Edit Fields)

1. Add **Edit Fields** → rename to `Store Route`
2. Add field:
   - Name: `route`
   - Value (Expression): `{{ $json.text.trim() }}`

### Step 5: Add the Switch node

1. Add **Switch** → rename to `Switch — Route`
2. Set **Mode**: Rules
3. Add 3 rules:

| Rule | Value 1 | Operation | Value 2 | Output Name |
|------|---------|-----------|---------|-------------|
| 1 | `{{ $json.route }}` | Equals | `refund` | refund |
| 2 | `{{ $json.route }}` | Equals | `order_status` | order_status |
| 3 | `{{ $json.route }}` | Equals | `support` | support |

### Step 6: Add the Specialist LLMs (one per branch)

For each Switch output, add a **Basic LLM Chain**:

**Refund Specialist:**
- Prompt (Expression):
  ```
  Subject: {{ $node['Input — Support Ticket'].json.ticket_subject }}
  Body:
  {{ $node['Input — Support Ticket'].json.ticket_body }}

  Write the reply:
  ```
- System:
  ```
  You are a customer support specialist for refunds.
  Write a short, professional reply.

  Rules:
  - Acknowledge the issue
  - Ask for any missing info (only if needed)
  - Explain next steps and expected timeline
  - Keep it under 120 words

  Output ONLY the reply.
  ```

**Order Status Specialist:**
- Same prompt structure
- System:
  ```
  You are a customer support specialist for order status.
  Write a short, professional reply.

  Rules:
  - Confirm you are checking the order
  - Ask for order number if missing
  - Provide what you can and what you still need
  - Keep it under 120 words

  Output ONLY the reply.
  ```

**Support Specialist:**
- Same prompt structure
- System:
  ```
  You are a general customer support specialist.
  Write a short, professional reply.

  Rules:
  - Clarify the problem
  - Ask 1–2 targeted questions (only if needed)
  - Offer a next step
  - Keep it under 120 words

  Output ONLY the reply.
  ```

### Step 7: Add Output nodes

For each specialist, add **Edit Fields** with:
- `reply`: `{{ $json.text }}`
- `route`: the branch name (e.g., `refund`)
- Enable **Keep Only Set**

### Step 8: Test

1. Click **Execute Workflow**
2. Check which branch received data (only one should have output)

::::

### Meet the Node: Switch

This pattern introduces a new node: **Switch**.

| Property | Description |
|----------|-------------|
| **Purpose** | Routes data to different branches based on conditions |
| **How it works** | Checks a value and sends data to the matching output |
| **Multiple outputs** | Each condition creates a separate output branch (solid line) |

**Configuration:**
1. **Mode:** Choose "Rules" for condition-based routing
2. **Data Type:** Usually "String" when routing on text values
3. **Value 1:** The field to check, e.g., `{{ $json.route }}`
4. **Operation:** Usually "Equals" for exact matching
5. **Value 2:** The value to match, e.g., `refund`

Add more rules to create more branches. The Switch sends data only to the branch where the condition matches.

**See the **Node Toolbox** appendix for full documentation.**

### What Problem This Solves

Different inputs need different handling. A support ticket about refunds should go to a refund specialist prompt, while an order status question goes elsewhere. Routing uses an LLM to classify the input, then a Switch node sends it down the right path.

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 300px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐     ┌──────────────────────┐     ┌───────────────┐     ┌────────────────┐
│   Manual Trigger │────▶│ Input — Support Ticket │────▶│  Router — Classify   │────▶│  Store Route  │────▶│ Switch — Route │───┐
└──────────────────┘     └────────────────────────┘     └──────────────────────┘     └───────────────┘     └────────────────┘   │
                                                                                                                                │
        ┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘
        │
        ├────▶┌─────────────────────────────┐────▶┌───────────────────────┐
        │     │   Refund Specialist         │     │   Output — Refund     │
        │     └─────────────────────────────┘     └───────────────────────┘
        │
        ├────▶┌─────────────────────────────┐────▶┌───────────────────────┐
        │     │   Order Status Specialist   │     │   Output — Order      │
        │     └─────────────────────────────┘     └───────────────────────┘
        │
        └────▶┌─────────────────────────────┐────▶┌───────────────────────┐
              │   Support Specialist        │     │   Output — Support    │
              └─────────────────────────────┘     └───────────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Run: Ticket Routing** | Manual Trigger | Starts the workflow |
| **Input — Support Ticket** | Set | Creates fields: `ticket_subject`, `ticket_body` |
| **Router — Choose Route** | Basic LLM Chain | Classifies ticket → outputs `text` (e.g., `refund`) |
| **Store Route** | Set | Saves `{{ $json.text.trim() }}` as `route` |
| **Switch — Route** | Switch | Checks `{{ $json.route }}` and sends to one branch |
| **Refund Specialist — Draft Reply** | Basic LLM Chain | Writes refund-specific reply → outputs `text` |
| **Order Status Specialist — Draft Reply** | Basic LLM Chain | Writes order status reply → outputs `text` |
| **Support Specialist — Draft Reply** | Basic LLM Chain | Writes general support reply → outputs `text` |
| **Output — Refund Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "refund"` |
| **Output — Order Status Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "order_status"` |
| **Output — Support Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "support"` |

### Prompts Used

**Router — Choose Route (the classifier):**
```
System: You route customer support tickets.

Choose exactly ONE route label, lowercase, no punctuation:
- refund
- order_status
- support

Return ONLY the label.
```

**Why this works:** The prompt forces clean output (lowercase, no punctuation, ONLY the label) so the Switch node can match exactly.

**Refund Specialist:**
```
System: You are a customer support specialist for refunds.
Write a short, professional reply.

Rules:
- Acknowledge the issue
- Ask for any missing info (only if needed)
- Explain next steps and expected timeline
- Keep it under 120 words

Output ONLY the reply.
```

**Order Status Specialist:**
```
System: You are a customer support specialist for order status.
Write a short, professional reply.

Rules:
- Confirm you are checking the order
- Ask for order number if missing
- Provide what you can and what you still need
- Keep it under 120 words

Output ONLY the reply.
```

**Support Specialist:**
```
System: You are a general customer support specialist.
Write a short, professional reply.

Rules:
- Clarify the problem
- Ask 1–2 targeted questions (only if needed)
- Offer a next step
- Keep it under 120 words

Output ONLY the reply.
```

### Key Expression: Accessing Earlier Node Data

The specialist nodes need the original ticket data, but they come after the Switch. They use this expression pattern:

```
{{ $node['Input — Support Ticket'].json.ticket_subject }}
{{ $node['Input — Support Ticket'].json.ticket_body }}
```

This accesses data from a specific earlier node by name. Useful when you need to "reach back" past intermediate nodes.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Support Ticket: { ticket_subject, ticket_body }
    ↓
Router LLM: { text: "refund" }
    ↓
Store Route: { text: "refund", route: "refund" }
    ↓
Switch: routes to ONE branch based on {{ $json.route }}
    ↓
[Refund Specialist]: { text: "Dear Jamie, I apologize..." }
    ↓                 (accesses original ticket via $node['Input — Support Ticket'])
Output — Refund Reply: { reply: "Dear Jamie, I apologize...", route: "refund" }
```

**Note:** The original `ticket_subject` and `ticket_body` are NOT passed through the Switch. The Specialist nodes access them using `$node['Input — Support Ticket'].json`.

### What to Observe

1. Click **Router — Choose Route** → see the classification in `text` (e.g., `refund`)
2. Click **Store Route** → see `route` field added
3. Click **Switch — Route** → only ONE output branch has data
4. Click the active specialist node → see the tailored reply

---

## Pattern 3: Parallelization

```
                        ┌─────────────────┐
                   ┌───▶│ LLM: Facts      │───┐
┌─────────────────┐│    └─────────────────┘   │    ┌─────────────────┐
│  Manual Trigger │┼───▶│ LLM: Sentiment  │───┼───▶│     Merge       │────▶ ...
└─────────────────┘│    └─────────────────┘   │    └─────────────────┘
                   └───▶│ LLM: Draft      │───┘
                        └─────────────────┘
```

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/03_parallelization.json
> ```
> 
> **Download:** {download}`03_parallelization.json <_static/workflows/03_parallelization.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Parallelization Practice"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Customer Email`
3. Add two String fields:

| Name | Value |
|------|-------|
| `email_subject` | `Can't access my account after password reset` |
| `email_body` | `Hello,\nI reset my password twice but I still can't log in. It says 'invalid token'.\nI'm trying to access my invoices before end of day.\nCan you help?\n— Sam` |

### Step 3: Connect to three parallel branches

From the Input node, drag THREE connections to create branches:

**Branch A — Extract Facts:**
1. Add **Basic LLM Chain** → rename to `Branch A — Extract Facts`
2. Prompt (Expression):
   ```
   Subject: {{ $json.email_subject }}
   Body:
   {{ $json.email_body }}

   JSON:
   ```
3. System:
   ```
   Extract key facts from the email.
   Return STRICT JSON with keys:
   customer_name, issue, deadline, requested_action, missing_info (array).
   Return JSON only.
   ```

**Branch B — Sentiment & Urgency:**
1. Add **Basic LLM Chain** → rename to `Branch B — Sentiment & Urgency`
2. Same prompt structure
3. System:
   ```
   Classify sentiment and urgency.
   Return STRICT JSON with keys:
   sentiment (positive|neutral|negative), urgency (low|medium|high), risk_flags (array).
   Return JSON only.
   ```

**Branch C — Draft Reply:**
1. Add **Basic LLM Chain** → rename to `Branch C — Draft Reply`
2. Prompt (Expression):
   ```
   Subject: {{ $json.email_subject }}
   Body:
   {{ $json.email_body }}

   Reply:
   ```
3. System:
   ```
   Draft a helpful customer support email reply.
   Rules:
   - Friendly and concise
   - Ask for missing info only if needed
   - Offer 1–2 concrete next steps
   - Under 140 words

   Output ONLY the reply text.
   ```

### Step 4: Add Store nodes after each branch

For each LLM, add **Edit Fields** with **Keep Only Set** enabled:
- Branch A: store `facts_json` = `{{ $json.text }}`
- Branch B: store `sentiment_json` = `{{ $json.text }}`
- Branch C: store `draft_reply` = `{{ $json.text }}`

### Step 5: Merge the branches

1. Add **Merge** node → rename to `Merge A+B`
   - Mode: **Combine by Position**
   - Connect Store Facts (input 1) and Store Sentiment (input 2)

2. Add another **Merge** → rename to `Merge (A+B)+C`
   - Connect Merge A+B (input 1) and Store Draft Reply (input 2)

### Step 6: Add the Finalize LLM

1. Add **Basic LLM Chain** → rename to `Finalize — One Improved Reply`
2. Prompt (Expression):
   ```
   facts_json:
   {{ $json.facts_json }}

   sentiment_json:
   {{ $json.sentiment_json }}

   draft_reply:
   {{ $json.draft_reply }}

   Final reply:
   ```
3. System:
   ```
   You are a senior support agent.
   You will receive:
   - facts_json (extracted facts)
   - sentiment_json (sentiment & urgency)
   - draft_reply (initial draft)

   Task:
   1) Improve the draft to match urgency and include any critical missing info questions.
   2) Keep it under 160 words.
   3) Output ONLY the final reply text.
   ```

### Step 7: Add Output

1. Add **Edit Fields** → rename to `Output — Final Reply`
2. Store `final_reply` = `{{ $json.text }}`
3. Enable **Keep Only Set**

### Step 8: Test

1. Click **Execute Workflow**
2. Check the merged output contains all three fields

::::

### Meet the Node: Merge

This pattern introduces a new node: **Merge**.

| Property | Description |
|----------|-------------|
| **Purpose** | Combines data from multiple branches into one |
| **Multiple inputs** | Receives data from 2+ parallel branches |
| **Output** | One combined item (or list) with all fields |

**Key Modes:**
| Mode | What it does |
|------|--------------|
| **Combine by Position** | Pairs items by index (1st + 1st, 2nd + 2nd). Best when each branch outputs one item. |
| **Combine by Fields** | Matches items by a common field value |
| **Append** | Puts all items into one list |

In this workflow, we use **Combine by Position** because each branch outputs exactly one item.

**See the **Node Toolbox** appendix for full documentation.**

### What Problem This Solves

Some tasks have independent parts that can be analyzed separately. Analyzing a customer email for facts, sentiment, and drafting a reply are independent—they don't need each other's results. Splitting them into branches keeps your workflow organized, and you can combine the results at the end for a final, informed response.

```{note}
**Not true parallel execution.** Despite the visual layout, n8n executes branches **sequentially** (A, then B, then C), not simultaneously. True parallel execution would require sub-workflows with webhook triggers — significantly more complex to set up. 

This pattern is still valuable for **code organization** and **clarity**, even without the speed benefit of real parallelization.
```

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 300px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐
│   Manual Trigger │────▶│ Input — Customer Email │───┬────▶┌─────────────────────────┐────▶┌───────────────────┐────┐
└──────────────────┘     └────────────────────────┘   │     │  Branch A — Facts       │     │    Store Facts    │    │
                                                      │     └─────────────────────────┘     └───────────────────┘    │
                                                      │                                                              │
                                                      ├────▶┌─────────────────────────┐────▶┌───────────────────┐    ├────▶┌─────────────┐────▶┌─────────────────┐────▶┌────────────────────┐────▶┌────────────────────────┐
                                                      │     │  Branch B — Sentiment   │     │  Store Sentiment  │    │     │  Merge A+B  │     │  Merge (A+B)+C  │     │ Finalize — Reply   │     │  Output — Final Reply  │
                                                      │     └─────────────────────────┘     └───────────────────┘    │     └─────────────┘     └─────────────────┘     └────────────────────┘     └────────────────────────┘
                                                      │                                                              │
                                                      └────▶┌─────────────────────────┐────▶┌───────────────────┐────┘
                                                            │  Branch C — Draft       │     │   Store Draft     │
                                                            └─────────────────────────┘     └───────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Run: Parallelization** | Manual Trigger | Starts the workflow |
| **Input — Customer Email** | Set | Creates fields: `email_subject`, `email_body` |
| **Branch A — Extract Facts** | Basic LLM Chain | Extracts facts as JSON → outputs `text` |
| **Store Facts** | Set | Saves `{{ $json.text }}` as `facts_json` |
| **Branch B — Sentiment & Urgency** | Basic LLM Chain | Analyzes sentiment → outputs `text` |
| **Store Sentiment** | Set | Saves `{{ $json.text }}` as `sentiment_json` |
| **Branch C — Draft Reply** | Basic LLM Chain | Drafts initial reply → outputs `text` |
| **Store Draft Reply** | Set | Saves `{{ $json.text }}` as `draft_reply` |
| **Merge A+B** | Merge | Combines `facts_json` + `sentiment_json` (mode: Combine by Position) |
| **Merge (A+B)+C** | Merge | Adds `draft_reply` to the combined data |
| **Finalize — One Improved Reply** | Basic LLM Chain | Uses all three fields → outputs `text` |
| **Output — Final Reply** | Set | Saves `{{ $json.text }}` as `final_reply` |

### Prompts Used

**Branch A — Extract Facts:**
```
System: Extract key facts from the email.
Return STRICT JSON with keys:
customer_name, issue, deadline, requested_action, missing_info (array).
Return JSON only.
```

**Branch B — Sentiment & Urgency:**
```
System: Classify sentiment and urgency.
Return STRICT JSON with keys:
sentiment (positive|neutral|negative), urgency (low|medium|high), risk_flags (array).
Return JSON only.
```

**Branch C — Draft Reply:**
```
System: Draft a helpful customer support email reply.
Rules:
- Friendly and concise
- Ask for missing info only if needed
- Offer 1–2 concrete next steps
- Under 140 words

Output ONLY the reply text.
```

**Finalize — One Improved Reply:**
```
System: You are a senior support agent.
You will receive:
- facts_json (extracted facts)
- sentiment_json (sentiment & urgency)
- draft_reply (initial draft)

Task:
1) Improve the draft to match urgency and include any critical missing info questions.
2) Keep it under 160 words.
3) Output ONLY the final reply text.
```

### Example JSON Outputs

**Branch A (facts_json):**
```json
{
  "customer_name": "Sam",
  "issue": "Can't log in after password reset, invalid token error",
  "deadline": "end of day",
  "requested_action": "Help accessing invoices",
  "missing_info": []
}
```

**Branch B (sentiment_json):**
```json
{
  "sentiment": "negative",
  "urgency": "high",
  "risk_flags": ["access issue", "time-sensitive", "repeated attempts"]
}
```

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Customer Email: { email_subject, email_body }
    ↓ ↓ ↓ (splits into 3 branches — executed sequentially)
    
Branch A: { text: '{"customer_name":"Sam",...}' }
    ↓
Store Facts: { facts_json: '{"customer_name":"Sam",...}' }

Branch B: { text: '{"sentiment":"negative","urgency":"high",...}' }
    ↓
Store Sentiment: { sentiment_json: '{"sentiment":"negative",...}' }

Branch C: { text: "Hi Sam, I understand..." }
    ↓
Store Draft Reply: { draft_reply: "Hi Sam, I understand..." }
    
    ↓ (merge all three)
After Merge: { facts_json, sentiment_json, draft_reply }
    ↓
Finalize LLM: { text: "Dear Sam, I sincerely apologize..." }
    ↓
Final Output: { final_reply: "Dear Sam, I sincerely apologize..." }
```

### What to Observe

1. Click **Input — Customer Email** → see the starting data
2. Click each **Branch** node → see different analyses running on the same input
3. Click **Merge (A+B)+C** → switch to JSON view to see all three fields combined
4. Click **Finalize** → see how the final LLM uses `{{ $json.facts_json }}`, `{{ $json.sentiment_json }}`, and `{{ $json.draft_reply }}`

::::{dropdown} 🔍 How does Merge combine branches? See the data flow
:color: info

The Merge node waits for all branches to complete, then combines their outputs into one item.

**Important:** Despite the visual layout, n8n runs branches **sequentially** (A → B → C), not in parallel.

```
                    ┌─────────────────────────────────┐
                    │        INPUT (same for all)     │
                    │ { "email_subject": "Help!",     │
                    │   "email_body": "Can't login" } │
                    └─────────────────────────────────┘
                                    │
                     ┌──────────────┼──────────────┐
                     ▼              ▼              ▼
            ┌──────────────┐ ┌──────────────┐ ┌──────────────┐
            │  Branch A    │ │  Branch B    │ │  Branch C    │
            │  (Facts)     │ │  (Sentiment) │ │  (Draft)     │
            │  runs 1st    │ │  runs 2nd    │ │  runs 3rd    │
            └──────────────┘ └──────────────┘ └──────────────┘
                     │              │              │
                     ▼              ▼              ▼
            ┌──────────────┐ ┌──────────────┐ ┌──────────────┐
            │ {            │ │ {            │ │ {            │
            │  "facts_json"│ │  "sentiment_ │ │  "draft_     │
            │  : "{...}"   │ │   json":     │ │   reply":    │
            │ }            │ │   "{...}"    │ │   "Hi Sam.." │
            │              │ │ }            │ │ }            │
            └──────────────┘ └──────────────┘ └──────────────┘
                     │              │              │
                     └──────────────┼──────────────┘
                                    ▼
                          ┌─────────────────┐
                          │      MERGE      │
                          │  (Combine by    │
                          │   Position)     │
                          └─────────────────┘
                                    │
                                    ▼
                    ┌─────────────────────────────────┐
                    │         MERGED OUTPUT           │
                    │ {                               │
                    │   "facts_json": "{...}",        │
                    │   "sentiment_json": "{...}",    │
                    │   "draft_reply": "Hi Sam..."    │
                    │ }                               │
                    │                                 │
                    │ ✅ All three fields combined!   │
                    └─────────────────────────────────┘
                                    │
                                    ▼
                          ┌─────────────────┐
                          │ Finalize LLM    │
                          │ uses ALL fields │
                          └─────────────────┘
```

**Key insight:** Even though execution is sequential, the pattern is valuable for organizing independent analyses. The Merge node combines all results so the final LLM can use all the information.

::::

---

## Pattern 4: Human-in-the-Loop

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Manual Trigger │────▶│   Edit Fields   │────▶│  LLM: Draft     │────▶│  ⏸️ Wait Node   │────▶│  Output: Send   │
└─────────────────┘     └─────────────────┘     └─────────────────┘     └─────────────────┘     └─────────────────┘
                                                                              │
                                                                        Human reviews
                                                                        & approves
```

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/04_human_in_the_loop.json
> ```
>
> **Download:** {download}`04_human_in_the_loop.json <_static/workflows/04_human_in_the_loop.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Human-in-the-Loop Practice"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Support Ticket`
3. Add three String fields:

| Name | Value |
|------|-------|
| `ticket_subject` | `Urgent: Need refund for duplicate charge` |
| `ticket_body` | `Hi, I was charged twice for order #12345. The second charge of $89.99 appeared yesterday. Please refund ASAP as this is causing overdraft fees. Thanks, Maria` |
| `customer_email` | `maria@example.com` |

### Step 3: Add the Draft LLM

1. Add **Basic LLM Chain** → rename to `Step 1 — Draft Response`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression):
     ```
     Subject: {{ $json.ticket_subject }}
     Body:
     {{ $json.ticket_body }}
     Customer: {{ $json.customer_email }}

     Draft the reply:
     ```
3. Add System Message:
   ```
   You are a customer support agent.
   Draft a professional email reply.

   Rules:
   - Acknowledge the issue with empathy
   - Confirm specific details (order #, amount)
   - Explain the refund process and timeline
   - Apologize for the inconvenience
   - Keep under 150 words

   Output ONLY the email body (no subject line).
   ```
4. Add your Chat Model

### Step 4: Store the draft

1. Add **Edit Fields** → rename to `Store Draft + Status`
2. Add five fields:
   - `draft_response`: `{{ $json.text }}`
   - `customer_email`: `{{ $node['Input — Support Ticket'].json.customer_email }}`
   - `ticket_subject`: `{{ $node['Input — Support Ticket'].json.ticket_subject }}`
   - `status`: `pending_approval`
   - `drafted_at`: `{{ $now.format('yyyy-MM-dd HH:mm') }}`

### Step 5: Add the Wait node

1. Add **Wait** → rename to `Wait — Human Approval`
2. Configure:
   - **Resume**: `On Webhook Call`
   - **Webhook Suffix** (in Options): `{{ $execution.id }}`

### Step 6: Add the Output node

1. Add **Edit Fields** → rename to `Output — Approved Response`
2. Add fields:
   - `final_response`: `{{ $node['Store Draft + Status'].json.draft_response }}`
   - `send_to`: `{{ $node['Store Draft + Status'].json.customer_email }}`
   - `subject`: `Re: {{ $node['Store Draft + Status'].json.ticket_subject }}`
   - `status`: `approved_and_sent`
3. Enable **Keep Only Set**

### Step 7: Test

1. Click **Execute Workflow** — it pauses at Wait
2. Click the **Wait node** → copy the **Test URL**
3. Open that URL in your browser → workflow resumes

::::

### Meet the Node: Wait

This pattern introduces a new node: **Wait**.

| Property | Description |
|----------|-------------|
| **Purpose** | Pauses workflow execution until resumed |
| **How it works** | Stops the workflow, waits for an external signal to continue |
| **Use cases** | Human approval, waiting for external events, timed delays |

**Resume Methods:**
| Method | Description |
|--------|-------------|
| **Webhook** | Resume when a specific URL is called (for approvals) |
| **On a specific date** | Resume at a scheduled time |
| **After time interval** | Resume after X minutes/hours |

In this workflow, we use **Webhook** mode: the workflow pauses until someone calls the approval URL.

**See the **Node Toolbox** appendix for full documentation.**

### What Problem This Solves

AI-generated content should often be reviewed before taking action. A support reply, a customer email, or a social media post might need human approval before sending. This pattern drafts the content with AI, then **pauses** for a human to review and approve.

**Why this matters:**
- Prevents embarrassing AI mistakes from reaching customers
- Gives humans control over final decisions
- Required in many industries (legal, healthcare, finance)

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 250px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐
│   Manual Trigger │────▶│ Input — Support Ticket │────▶│  Step 1 — Draft        │────▶│  Store Draft + Status  │
└──────────────────┘     └────────────────────────┘     └────────────────────────┘     └────────────────────────┘
                                                                                                │
                                                                                                ▼
                                                                                       ┌────────────────────────┐
                                                                                       │  ⏸️ Wait — Approval    │
                                                                                       │     (pauses here)      │
                                                                                       └────────────────────────┘
                                                                                                │
                                                                                          Human approves
                                                                                                │
                                                                                                ▼
                                                                                       ┌────────────────────────┐
                                                                                       │  Output — Approved     │
                                                                                       │  Response              │
                                                                                       └────────────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Run: Human Approval** | Manual Trigger | Starts the workflow |
| **Input — Support Ticket** | Set | Creates fields: `ticket_subject`, `ticket_body`, `customer_email` |
| **Step 1 — Draft Response** | Basic LLM Chain | Generates reply draft → outputs `text` |
| **Store Draft + Status** | Set | Saves draft, sets `status: "pending_approval"`, and records `drafted_at` timestamp |
| **Wait — Human Approval** | Wait | Pauses workflow until approval webhook is called |
| **Output — Approved Response** | Set | Formats final output with `status: "approved_and_sent"` |

### Prompt Used

**Step 1 — Draft Response:**
```
System: You are a customer support agent.
Draft a professional email reply.

Rules:
- Acknowledge the issue with empathy
- Confirm specific details (order #, amount)
- Explain the refund process and timeline
- Apologize for the inconvenience
- Keep under 150 words

Output ONLY the email body (no subject line).
```

### How to Resume the Wait Node

The Wait node in **webhook mode** creates a unique URL for this execution.

**To test (resume manually):**
1. Run the workflow — it pauses at the Wait node
2. Click the **Wait node**
3. Copy the **Test URL** from the right panel
4. Open that URL in your browser — workflow resumes

**In production:** Send the webhook URL to Slack/Email with an "Approve" button. When clicked, the workflow continues.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Support Ticket: { ticket_subject, ticket_body, customer_email }
    ↓
Draft LLM: { text: "Dear Maria, I sincerely apologize..." }
    ↓
Store Draft: { 
  draft_response: "Dear Maria, I sincerely apologize...",
  customer_email: "maria@example.com",
  ticket_subject: "Urgent: Need refund...",
  status: "pending_approval",
  drafted_at: "2025-01-25 14:30"
}
    ↓
⏸️ PAUSE — workflow waits here
    ↓
(human opens Test URL in browser)
    ↓
Output: { 
  final_response: "Dear Maria, I sincerely apologize...",
  send_to: "maria@example.com",
  subject: "Re: Urgent: Need refund...",
  status: "approved_and_sent"
}
```

**Note:** The `drafted_at` field uses `{{ $now.format('yyyy-MM-dd HH:mm') }}` — a built-in expression to record when the draft was created. This helps reviewers know if a draft is fresh or has been waiting for hours.

### What to Observe

1. Click **Step 1 — Draft Response** → see the AI-generated reply
2. Click **Store Draft + Status** → see `status: "pending_approval"` and `drafted_at` timestamp
3. Notice the workflow **pauses** at the Wait node
4. Copy Test URL, open in browser → workflow continues
5. Click **Output** → see `status: "approved_and_sent"`

### Production Integration Ideas

| Integration | How to implement |
|-------------|-----------------|
| **Slack approval** | Before Wait: send draft to Slack with interactive buttons. Button calls webhook. |
| **Email approval** | Send draft email with "Approve" link that calls the webhook URL |
| **Dashboard** | Store draft in database, show in admin panel with Approve/Reject buttons |

The Wait node's webhook URL can include the execution ID (`{{ $execution.id }}`) to route approvals to the correct paused workflow.

---

## AI Agent Node (Minimal Intro)

The **AI Agent** node is a workflow step that can **decide what to do next** instead of following a single fixed prompt.

- **Tools:** optional actions the agent can call (e.g., calculator, HTTP request, search). Think: "abilities".
- **Memory:** optional context the agent can carry across steps so it stays consistent.
- **Loop (plan → act → check):** the agent can iterate: decide a plan, use a tool (act), check results, and stop when done.

**Tip:** Start simple—use **one tool** (or none) and keep runs in **manual test mode** to avoid accidental costs.

See the **First AI Agent** chapter for a hands-on walkthrough.

---

## Pattern Summary

| Pattern | When to use | Key nodes |
|---------|-------------|------------|
| **Prompt Chaining** | Complex tasks that benefit from step-by-step refinement | Multiple LLM Chains in sequence |
| **Routing** | Different handling based on input type | LLM (classifier) + Switch |
| **Parallelization** | Independent analyses that can run simultaneously | Multiple branches + Merge |
| **Human-in-the-Loop** | AI output needs human approval before action | LLM + Wait (webhook) |

---

## Tips for Building Your Own Workflows

1. **Start simple.** Build one node at a time and test frequently.

2. **Use descriptive node names.** "Step 1: Create Outline" is better than "LLM Chain."

3. **Pin data often.** After any successful LLM call, pin the result before working on the next node.

4. **Check the Output panel.** Switch between Table and JSON views to understand your data.

5. **Use Set nodes as checkpoints.** Save intermediate results with clear field names.

6. **Test routing with different inputs.** Make sure all branches work, not just the happy path.